In [166]:
import queue
from experiment_information import *
from experiments.sweep_generalization_error import *
from data import *
from _version import __version__
import pandas as np
import numpy as np
import logging
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [167]:

from _version import __version__
df_experiments = None
df_state_evolution = None
df_erm = None
logger = logging.getLogger()
with DatabaseHandler(logger,"experiments/experiments.db") as dbHandler:
    df_experiments = dbHandler.get_experiments()
    df_state_evolution = dbHandler.get_state_evolutions()
    df_erm = dbHandler.get_erms()
    # delete incomplete experiments (bad, deletes running experiments...)
    dbHandler.delete_incomplete_experiments()
print("Current code version, ", __version__)

Current code version,  22


In [168]:
# get the completed experiments with the current code version ordered by date
version_choice = __version__
# version_choice = "13"
df_experiments = df_experiments[(df_experiments["completed"]==True) & (df_experiments["code_version"]==version_choice)]
df_experiments = df_experiments.sort_values(by="date",ascending=False)
df_experiments.head(2)

,experiment_id,experiment_name,duration,code_version,date,state_evolution_repetitions,erm_repetitions,alphas,epsilons,lambdas,taus,d,erm_methods,completed
24,059bb116-5415-4935-8622-01ca7a842ed7,Default Experiment,30.736378,22,2023-04-06 09:37:30,0,5,"[0.2, 0.8, 1.3, 1.7, 2.0, 2.5]","[0.0, 0.2]",[1e-05],[2],50,"[""sklearn""]",1
23,8df18cc3-b1a9-4c17-9109-07603babd06c,Default Experiment,195.726795,22,2023-04-06 09:33:25,0,5,"[0.2, 0.8, 1.3, 1.7, 2.0, 2.5]","[0.0, 0.2]",[1e-05],[2],500,"[""sklearn""]",1


In [169]:
# extract and print the top experiment_id
experiment_id = df_experiments.iloc[0]["experiment_id"]
print(experiment_id)

059bb116-5415-4935-8622-01ca7a842ed7


In [170]:
state_evolution = df_state_evolution[df_state_evolution["experiment_id" ] == experiment_id]
erm = df_erm[df_erm["experiment_id" ] == experiment_id]
# gd = erm[erm["chosen_minimizer"] == "gd"]
# gd = erm[erm["chosen_minimizer"] == "L-BFGS-B"]
# gd = erm[erm["chosen_minimizer"] == "logistic"]
gd = erm[erm["chosen_minimizer"] == "sklearn"]
# set alpha, epsilon, tau, lam as index in gd and state_evolution
gd = gd.set_index(["alpha","epsilon","tau","lam"])
state_evolution = state_evolution.set_index(["alpha","epsilon","tau","lam"])
# from state_evolution keep generalization_error, sigma, q, m, cosb
state_evolution = state_evolution[["generalization_error","sigma","q","m","cosb","training_error"]]
# add a suffix _state_evolution to the columns
state_evolution.columns = [col+"_state_evolution" for col in state_evolution.columns]
# from gd keep generalization_error_erm	generalization_error_overlap Q	rho	m	cosb
gd = gd[["generalization_error_erm","generalization_error_overlap","Q","rho","m","cosb", "training_error"]]
# add a suffix _erm to the columns
gd.columns = [col+"_erm" for col in gd.columns]
# join the two dataframes
df = state_evolution.join(gd)
# reorder the columns
df = df[['generalization_error_state_evolution', 'generalization_error_erm_erm', 'generalization_error_overlap_erm', 'sigma_state_evolution', 'q_state_evolution','Q_erm','m_state_evolution','m_erm','cosb_state_evolution','cosb_erm', 'rho_erm',"training_error_state_evolution","training_error_erm"]]
# average all quantities over the index and obtain also the standard deviation
df = df.groupby(level=[0,1,2,3]).agg([np.mean,np.std])
# display all columns
pd.set_option('display.max_columns', None)
df

Empty DataFrame
Columns: [(generalization_error_state_evolution, mean), (generalization_error_state_evolution, std), (generalization_error_erm_erm, mean), (generalization_error_erm_erm, std), (generalization_error_overlap_erm, mean), (generalization_error_overlap_erm, std), (sigma_state_evolution, mean), (sigma_state_evolution, std), (q_state_evolution, mean), (q_state_evolution, std), (Q_erm, mean), (Q_erm, std), (m_state_evolution, mean), (m_state_evolution, std), (m_erm, mean), (m_erm, std), (cosb_state_evolution, mean), (cosb_state_evolution, std), (cosb_erm, mean), (cosb_erm, std), (rho_erm, mean), (rho_erm, std), (training_error_state_evolution, mean), (training_error_state_evolution, std), (training_error_erm, mean), (training_error_erm, std)]
Index: []

In [171]:
# plot training_error_erm in gd dataframe
# set the title to "Training Error"
training_error = gd["training_error_erm"]
# average over alpha
training_error = training_error.groupby(level=[0,1])
training_error = training_error.agg([np.mean,np.std])
training_error

mean       std
alpha epsilon                    
0.2   0.0      0.000000  0.000000
      0.2      0.000000  0.000000
0.8   0.0      0.000000  0.000000
      0.2      0.000000  0.000000
1.3   0.0      0.000000  0.000000
      0.2      0.000000  0.000000
1.7   0.0      0.000000  0.000000
      0.2      0.032941  0.045867
2.0   0.0      0.026000  0.058138
      0.2      0.126000  0.027928
2.5   0.0      0.168000  0.046303
      0.2      0.187200  0.046853

In [172]:
# keep only the alpha in the index
# plot generalization_error_state_evolution vs alpha
# set the title to "Generalization Error"
df["generalization_error_state_evolution"]["mean"].unstack(level=1).plot()
plt.title("Generalization Error")
df["generalization_error_erm_erm"]["mean"].unstack(level=1).plot()
# set the title to "Generalization Error"
plt.title("Generalization Error")
# plot generalization_error_overlap_erm vs alpha
# set the title to "Generalization Error Overlap" 
df["generalization_error_overlap_erm"]["mean"].unstack(level=1).plot()
plt.title("Generalization Error Overlap")


TypeError: no numeric data to plot

In [ ]:
# plot the training_error vs alpha
df["training_error_erm"]["mean"].unstack(level=1).plot()
# set the title to "Training Error"
plt.title("Training Error")
df["training_error_state_evolution"]["mean"].unstack(level=1).plot()
# set the title to "Training Error"
plt.title("Training Error")

TypeError: no numeric data to plot

,mean,std
alpha,,
0.2,0.00000,0.000000
0.8,0.00000,0.000000
1.3,0.00000,0.000000
1.7,0.00000,0.000000
2.0,0.00000,0.000000
2.5,0.14868,0.010506


In [ ]:
# Create a SweepExperiment
experiment = get_default_experiment()
# use json dump to save the experiment parameters
with open("sweep_experiment.json","w") as f:
    # use the NumpyEncoder to encode numpy arrays
    json.dump(experiment.__dict__,f,cls=NumpyEncoder)